In [1]:
import dask
import xarray as xr
import numpy as np
import pandas as pd
import yaml
from rhg_compute_tools import kubernetes as rhgk
from dodola.services import xesmf_regrid
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import dask.distributed as dd
import dask_gateway
from tqdm import tqdm

/tmp/ipykernel_33229/3093552774.py:6: DeprecationWarning: rhg_compute_tools and the associated command-line application rctools are
deprecated on catalyst clusters. use catalyst_compute_tools instead.
See https://gitlab.com/crs-catalyst/infrastructure/catalyst_compute_tools/-/issues/1
  from rhg_compute_tools import kubernetes as rhgk


In [2]:
client, cluster = rhgk.get_standard_cluster()
cluster.scale(300)

In [54]:
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/jhub.3fbc8e3e769d43dd8efaf0262fb76d88/status,


In [58]:
wtype = 'land' # type of pixel weighting for global means

In [59]:
def read_weights(da, wtype = 'land'):

    assert wtype in ['land', 'area']
    
    if wtype == 'land':
        wf = "gs://support-data-cc7330d0/public_datasets/spatial/exposure/GLOBAL/land_water_area/gpw-v4-land-water-area-rev11_landareakm_30_sec_tif/derived_datasets/coarsened/gpw-v4-land-water-area-rev11_landareakm_0.25degree.zarr"

        wda = xr.open_zarr(wf, chunks=None)
        wda = wda.load().land_area
        
    else:
        
        lat_weights = np.cos(da['lat'] * np.pi / 180.0)
        wda = xr.ones_like(da) * lat_weights
    
    return wda

In [60]:
with open('/home/jovyan/repositories/downscaleCMIP6/notebooks/downscaling_pipeline/post_processing_and_delivery/data_paths.yaml', 'r') as f:
    fps = yaml.load(f, yaml.Loader)

In [61]:
def cyclic_lon(da):
    """
    [0, 360] - > [-180, 180]
    """
    return da.assign_coords(
        {"lon": xr.where(da.lon > 180, da.lon - 360, da.lon)}
    ).sortby("lon")

In [62]:
def read_data_temporal_slice(
    model=model,
    scenario=fut_scenario,
    step_label="downscaled_delivered",
    pix=None,
    var="tasmax",
):
    assert step_label in ['clean','downscaled_delivered']
    assert var in ['tasmax','pr']
    if step_label == 'clean':
        if 'ssp' in scenario:
            time_slice = slice("2080-01-01", "2100-12-31")
        elif scenario=='historical':
            time_slice = slice("1995-01-01", "2014-12-31")

        fp = fps[f"{model}-{var}"][scenario][step_label]
        print(f'reading zarr {fp}')
        da = xr.open_zarr(fp)[var]
        if pix is not None:
            da = da.isel(lat=pix["lat"], lon=pix["lon"], drop=True)

        da = da.sel(time=time_slice)

        da = regrid_to_downscaled_resolution(da=da)
    else:
        var_diag = var if var != 'pr' else 'precip'
        da_var_diag = 'annual_average_tasmax' if var == 'tasmax' else 'total_annual_precip'
        da = xr.open_zarr(fps[f"{model}-{var}"][scenario]['downscaled_delivered'].replace('outputs', f'diagnostics/RELEASE-v1.1/annual-{var_diag}-diagnostics'))[da_var_diag] 
        
        if 'ssp' in scenario:
            da = da.sel(year=slice(2080, 2100))
        else:
            da = da.sel(year=slice(1995, 2014))
    return da

In [63]:
def annual_sums(da):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        return da.groupby("time.year").sum()

In [64]:
def annual_mean(da):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):
        return da.groupby("time.year").mean()

In [65]:
def regrid_to_downscaled_resolution(da, method="nearest_s2d"):
    """
    regrid a lat/lon dataarray to the downscaling resolution using dodola's `xesmf_regrid`.
    """
    domain_ds = xr.open_zarr("gs://support-c23ff1a3/domain.0p25x0p25.zarr", chunks=None)
    da = xesmf_regrid(
        x=xr.Dataset({"da": da}),
        domain=domain_ds,
        method=method,
        astype=np.float32,
        add_cyclic=None,
        keep_attrs=True,
    )["da"]
    return da

In [66]:
def global_average_by_year(da, var, weights, step_label):
    if step_label == 'clean':
        da = annual_mean(da) if var == 'tasmax' else annual_sums(da)
    if weights is not None:
        da = da.weighted(weights)
    da = da.mean(['lat','lon'])
    return da

In [67]:
def average_across_years(da):
    da = da.mean('year')
    return da

In [68]:
def do_global_computation(model, var, step_label, weights, **kwargs):
    
    print('historical')
    
    hist_da = read_data_temporal_slice(var=var, scenario='historical', model=model, step_label=step_label, **kwargs)
    hist_da = global_average_by_year(hist_da, var=var, weights=weights, step_label=step_label)
    hist_da = average_across_years(hist_da)

    both_scens_results = []
    for scenario in ['ssp245','ssp370']:
        print(scenario)
        da = read_data_temporal_slice(var=var, scenario=scenario, model=model, step_label=step_label, **kwargs)
        da = global_average_by_year(da, var=var, weights=weights, step_label=step_label)
        da = average_across_years(da)
    
        if var == 'pr':
            trend = da/hist_da
        else:
            trend = da - hist_da

        trend = trend.expand_dims(dict(scenario=[scenario], model=[model], step=['clean' if step_label == 'clean' else 'biascorrected_downscaled']))

        trend.name = 'da'
        
        both_scens_results.append(trend)
    
    out = xr.combine_by_coords(both_scens_results)
    
    return out

In [69]:
weights_template = xr.open_zarr('gs://downscaled-288ec5ac/outputs/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/day/pr/v1.1.zarr', chunks=None).pr.isel(time=0, drop=True).load()

In [70]:
weights = read_weights(weights_template, wtype)

In [71]:
# time it took without computations : 31 secs (mostly due to regridding)
# import time
# t0 = time.time()
# test_metadata = do_global_computation(var='pr', step_label='clean', model='NorESM2-LM', weights=read_weights(weights_template, 'land'))
# t1 = time.time()
# print(t1-t0)

In [72]:
# time it took to compute : around 42 secs
# import time
# t0 = time.time()
# test_result = test_metadata.compute()
# t1 = time.time()
# print(t1-t0)

## Gathering the clean data -- needs a lot of computation as it's not ready yet. 

### precip clean

In [99]:
# we're first gathering all the graph, and then computing, to leverage a large cluster.
precip_metadata = []
for i,k in tqdm(enumerate(list(fps.keys()))):
    if k[-3:] == '-pr':
        if all(scenario_key in fps[k] for scenario_key in ['ssp245','ssp370','historical']):
            model = k.replace('-pr', '')
            print(model)
            precip_metadata.append(do_global_computation(var='pr', step_label='clean', model=model, weights=weights))

0it [00:00, ?it/s]

ACCESS-CM2
historical
reading zarr gs://clean-b1dbca25/cmip6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/day/pr/gn/v20191108.zarr
ssp245
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp245/r1i1p1f1/day/pr/gn/v20191108.zarr
ssp370
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp370/r1i1p1f1/day/pr/gn/v20191108.zarr


1it [00:22, 22.77s/it]

ACCESS-ESM1-5
historical
reading zarr gs://clean-b1dbca25/cmip6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/day/pr/gn/v20191115.zarr
ssp245
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp245/r1i1p1f1/day/pr/gn/v20191115.zarr
ssp370
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp370/r1i1p1f1/day/pr/gn/v20191115.zarr


4it [00:45, 10.34s/it]

...

72it [06:54,  4.62s/it]

NorESM2-MM
historical
reading zarr gs://clean-b1dbca25/cmip6/CMIP/NCC/NorESM2-MM/historical/r1i1p1f1/day/pr/gn/v20191108.zarr
ssp245
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/NCC/NorESM2-MM/ssp245/r1i1p1f1/day/pr/gn/v20191108.zarr
ssp370
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/NCC/NorESM2-MM/ssp370/r1i1p1f1/day/pr/gn/v20191108.zarr


75it [07:17,  5.28s/it]

UKESM1-0-LL
historical
reading zarr gs://clean-b1dbca25/cmip6/CMIP/MOHC/UKESM1-0-LL/historical/r1i1p1f2/day/pr/gn/v20190627.zarr
ssp245
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/MOHC/UKESM1-0-LL/ssp245/r1i1p1f2/day/pr/gn/v20190715.zarr
ssp370
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/MOHC/UKESM1-0-LL/ssp370/r1i1p1f2/day/pr/gn/v20190726.zarr


80it [07:39,  5.75s/it]


In [100]:
precip_metadata_da = xr.combine_by_coords(precip_metadata)

In [101]:
precip_results_da = precip_metadata_da.compute()

In [102]:
precip_results_da.to_zarr('gs://downscaled-288ec5ac/diagnostics/paper_figures/global_means_gdpcir_versus_source/precip_clean.zarr')

### tasmax clean

In [103]:
# we're first gathering all the graph, and then computing, to leverage a large cluster.
tasmax_metadata = []
for i,k in tqdm(enumerate(list(fps.keys()))):
    if '-tasmax' in k:
        if all(scenario_key in fps[k] for scenario_key in ['ssp245','ssp370','historical']):
            model = k.replace('-tasmax', '')
            print(model)
            tasmax_metadata.append(do_global_computation(var='tasmax', step_label='clean', model=model, weights=weights))

0it [00:00, ?it/s]

ACCESS-CM2
historical
reading zarr gs://clean-b1dbca25/cmip6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/day/tasmax/gn/v20191108.zarr
ssp245
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp245/r1i1p1f1/day/tasmax/gn/v20191108.zarr
ssp370
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp370/r1i1p1f1/day/tasmax/gn/v20191108.zarr


2it [00:22, 11.43s/it]

ACCESS-ESM1-5
historical
reading zarr gs://clean-b1dbca25/cmip6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/day/tasmax/gn/v20191115.zarr
ssp245
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp245/r1i1p1f1/day/tasmax/gn/v20191115.zarr
ssp370
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp370/r1i1p1f1/day/tasmax/gn/v20191115.zarr


5it [00:45,  8.89s/it]

...

73it [08:08,  5.90s/it]

NorESM2-MM
historical
reading zarr gs://clean-b1dbca25/cmip6/CMIP/NCC/NorESM2-MM/historical/r1i1p1f1/day/tasmax/gn/v20191108.zarr
ssp245
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/NCC/NorESM2-MM/ssp245/r1i1p1f1/day/tasmax/gn/v20191108.zarr
ssp370
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/NCC/NorESM2-MM/ssp370/r1i1p1f1/day/tasmax/gn/v20191108.zarr


76it [08:32,  6.36s/it]

UKESM1-0-LL
historical
reading zarr gs://clean-b1dbca25/cmip6/CMIP/MOHC/UKESM1-0-LL/historical/r1i1p1f2/day/tasmax/gn/v20190627.zarr
ssp245
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/MOHC/UKESM1-0-LL/ssp245/r1i1p1f2/day/tasmax/gn/v20190715.zarr
ssp370
reading zarr gs://clean-b1dbca25/cmip6/ScenarioMIP/MOHC/UKESM1-0-LL/ssp370/r1i1p1f2/day/tasmax/gn/v20190726.zarr


80it [08:54,  6.68s/it]


In [104]:
tasmax_metadata_da = xr.combine_by_coords(tasmax_metadata)

In [105]:
tasmax_results_da = tasmax_metadata_da.compute()

In [106]:
tasmax_results_da.to_zarr('gs://downscaled-288ec5ac/diagnostics/paper_figures/global_means_gdpcir_versus_source/tasmax_clean.zarr')

## Gathering the downscaled data from the already computed diagnostics -- fast

In [108]:
MODELS_BY_LICENSE = {
    "cc0": [
        "FGOALS-g3",
        "INM-CM4-8",
        "INM-CM5-0",
    ],
    "cc_by": [
        "ACCESS-CM2",
        "ACCESS-ESM1-5",
        "BCC-CSM2-MR",
        "CMCC-CM2-SR5",
        "CMCC-ESM2",
        "EC-Earth3-AerChem",
        "EC-Earth3-CC",
        "EC-Earth3-Veg-LR",
        "EC-Earth3-Veg",
        "EC-Earth3",
        "GFDL-CM4",
        "GFDL-ESM4",
        "HadGEM3-GC31-LL",
        "MIROC-ES2L",
        "MIROC6",
        "MPI-ESM1-2-HR",
        "MPI-ESM1-2-LR",
        "NESM3",
        "NorESM2-LM",
        "NorESM2-MM",
        "UKESM1-0-LL",
    ],
    "cc_by_sa": [
        "CanESM5",
    ]
}

ALL_MODELS = sum(MODELS_BY_LICENSE.values(), [])

### precip downscaled

In [109]:
# we're first gathering all the graph, and then computing, to leverage a large cluster.
precip_metadata_gdpcir = []
for i,k in tqdm(enumerate(list(fps.keys()))):
    if k[-3:] == '-pr':
        if all(scenario_key in fps[k] for scenario_key in ['ssp245','ssp370','historical']):
            model = k.replace('-pr', '')
            if model not in ALL_MODELS:
                continue
            print(model)
            precip_metadata_gdpcir.append(do_global_computation(var='pr', step_label='downscaled_delivered', model=model, weights=weights))

0it [00:00, ?it/s]

ACCESS-CM2
historical
ssp245
ssp370


4it [00:01,  3.60it/s]

ACCESS-ESM1-5
historical
ssp245
ssp370
BCC-CSM2-MR
historical


8it [00:01,  7.34it/s]

...

72it [00:12,  3.17it/s]

NorESM2-MM
historical
ssp245
ssp370


75it [00:14,  2.45it/s]

UKESM1-0-LL
historical
ssp245
ssp370


80it [00:17,  4.61it/s]


In [110]:
precip_metadata_da_gdpcir = xr.combine_by_coords(precip_metadata_gdpcir)

In [111]:
precip_results_da_gdpcir = precip_metadata_da_gdpcir.compute()

In [112]:
precip_results_da_gdpcir.to_zarr('gs://downscaled-288ec5ac/diagnostics/paper_figures/global_means_gdpcir_versus_source/precip_gdpcir.zarr')

### tasmax downscaled

In [114]:
# we're first gathering all the graph, and then computing, to leverage a large cluster.
tasmax_metadata_gdpcir = []
for i,k in tqdm(enumerate(list(fps.keys()))):
    if '-tasmax' in k:
        if all(scenario_key in fps[k] for scenario_key in ['ssp245','ssp370','historical']):
            model = k.replace('-tasmax', '')
            if model not in ALL_MODELS:
                continue
            print(model)
            tasmax_metadata_gdpcir.append(do_global_computation(var='tasmax', step_label='downscaled_delivered', model=model, weights=weights))

2it [00:00, 12.40it/s]

ACCESS-CM2
historical
ssp245
ssp370
ACCESS-ESM1-5
historical


5it [00:00, 15.85it/s]

ssp245
ssp370
BCC-CSM2-MR
historical
ssp245
ssp370


9it [00:03,  2.10it/s]

...

73it [00:37,  2.16it/s]

NorESM2-MM
historical
ssp245
ssp370


76it [00:39,  1.91it/s]

UKESM1-0-LL
historical
ssp245
ssp370


80it [00:42,  1.89it/s]


In [115]:
tasmax_metadata_da_gdpcir = xr.combine_by_coords(tasmax_metadata_gdpcir)

In [116]:
tasmax_results_da_gdpcir = tasmax_metadata_da_gdpcir.compute()

In [117]:
tasmax_results_da_gdpcir.to_zarr('gs://downscaled-288ec5ac/diagnostics/paper_figures/global_means_gdpcir_versus_source/tasmax_gdpcir.zarr')

In [122]:
client.close(), cluster.close()

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:1014: RuntimeWarning: coroutine 'rpc.close_rpc' was never awaited
  self.scheduler_comm.close_rpc()


(None, None)